In [1]:
import numpy as np
import pandas as pd

from astropy import coordinates
from astropy.coordinates import match_coordinates_sky
import astropy.units as u
import astroquery
from astroquery.irsa import Irsa
from astroquery.vizier import Vizier
from astropy.table import Table, join

from astropy.units import Quantity
from astroquery.gaia import Gaia

Irsa.ROW_LIMIT = -1
Vizier.ROW_LIMIT = -1

from astropy.io import ascii as asci

import matplotlib.pyplot as plt

# Suppress warnings. Comment this out if you wish to see the warning messages
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

Created TAP+ (v1.0.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: False
	Port: 80
	SSL Port: 443


# Get the data

2MASS => J, H K, angular resolution ~4"

WISE => 3.4, 4.6, 12, and 22 μm (W1, W2, W3, W4) with an angular resolution of 6.1", 6.4", 6.5", & 12.0" 

GALEX imaging => Five imaging surveys in a Far UV band (1350-1750Å) and Near UV band (1750-2800Å) with 6-8 arcsecond resolution (80% encircled energy) and 1 arcsecond astrometry, and a cosmic UV background map.

In [2]:
from astroquery.gaia import Gaia
tables = Gaia.load_tables(only_names=True)
for table in (tables):
    print (table.get_qualified_name())

Retrieving tables...
Parsing tables...
Done.
external.external.gaiadr2_geometric_distance
public.public.dual
public.public.hipparcos
public.public.hipparcos_newreduction
public.public.hubble_sc
public.public.igsl_source
public.public.igsl_source_catalog_ids
public.public.tycho2
tap_schema.tap_schema.columns
tap_schema.tap_schema.key_columns
tap_schema.tap_schema.keys
tap_schema.tap_schema.schemas
tap_schema.tap_schema.tables
gaiadr1.gaiadr1.aux_qso_icrf2_match
gaiadr1.gaiadr1.ext_phot_zero_point
gaiadr1.gaiadr1.allwise_best_neighbour
gaiadr1.gaiadr1.allwise_neighbourhood
gaiadr1.gaiadr1.gsc23_best_neighbour
gaiadr1.gaiadr1.gsc23_neighbourhood
gaiadr1.gaiadr1.ppmxl_best_neighbour
gaiadr1.gaiadr1.ppmxl_neighbourhood
gaiadr1.gaiadr1.sdss_dr9_best_neighbour
gaiadr1.gaiadr1.sdss_dr9_neighbourhood
gaiadr1.gaiadr1.tmass_best_neighbour
gaiadr1.gaiadr1.tmass_neighbourhood
gaiadr1.gaiadr1.ucac4_best_neighbour
gaiadr1.gaiadr1.ucac4_neighbourhood
gaiadr1.gaiadr1.urat1_best_neighbour
gaiadr1.gaiadr

In [4]:
#obj = ["3C 454.3", 343.49062, 16.14821, 1.0]
obj = ["PKS J0006-0623", 1.55789, -6.39315, 1]
#obj = ["M87", 187.705930, 12.391123, 1.0]
#### name, ra, dec, radius of cone (in deg)

obj_name = obj[0]
obj_ra   = obj[1]
obj_dec  = obj[2]
cone_radius  = obj[3]

In [49]:
obj_coord = coordinates.SkyCoord(ra=obj_ra, dec=obj_dec, unit=(u.deg, u.deg), frame="icrs")

# Try GAIA with 2MASS

- in gaiadr1 and gaiadr2 they also provide 2mass, allwise, sdss "best neigbour" pairs 
- catalogs provided by GAIA:
    - gaiadr1.gaiadr1.allwise_original_valid
    - gaiadr1.gaiadr1.gsc23_original_valid
    - gaiadr1.gaiadr1.ppmxl_original_valid
    - gaiadr1.gaiadr1.sdssdr9_original_valid
    - gaiadr1.gaiadr1.tmass_original_valid
    - gaiadr1.gaiadr1.ucac4_original_valid
    - gaiadr1.gaiadr1.urat1_original_valid

In [64]:
# cmd = "SELECT * \
# FROM gaiadr2.gaia_source \
# WHERE CONTAINS(POINT('ICRS',gaiadr2.gaia_source.ra,gaiadr2.gaia_source.dec), \
# CIRCLE('ICRS'," + str(obj_ra) + "," + str(obj_dec) + "," + str(cone_radius) + "))=1;"

cmd = "SELECT * FROM gaiadr2.gaia_source AS g, \
gaiadr2.tmass_best_neighbour AS tbest, \
gaiadr1.tmass_original_valid AS tmass \
WHERE g.source_id = tbest.source_id AND tbest.tmass_oid = tmass.tmass_oid AND CONTAINS(POINT('ICRS',g.ra,g.dec),\
CIRCLE('ICRS'," + str(obj_ra) + "," + str(obj_dec) + "," + str(cone_radius) + "))=1;"

print(cmd)


job = Gaia.launch_job_async(cmd, dump_to_file=True)
print (job)


# GAIA

SELECT * FROM gaiadr2.gaia_source AS g, gaiadr2.tmass_best_neighbour AS tbest, gaiadr1.tmass_original_valid AS tmass WHERE g.source_id = tbest.source_id AND tbest.tmass_oid = tmass.tmass_oid AND CONTAINS(POINT('ICRS',g.ra,g.dec),CIRCLE('ICRS',1.55789,-6.39315,1))=1;
Jobid: 1542987284378O
Phase: COMPLETED
Owner: None
Output file: async_20181123123444.vot
Results: None


In [65]:
r = job.get_results()
print(len(r['source_id']))

4207


In [66]:
print(r['phot_g_mean_mag', 'phot_bp_mean_mag', 'phot_rp_mean_mag', 'j_m', 'h_m', 'ks_m', 'tmass_oid']) 

phot_g_mean_mag phot_bp_mean_mag phot_rp_mean_mag  j_m    h_m    ks_m  tmass_oid
      mag             mag              mag         mag    mag    mag            
--------------- ---------------- ---------------- ------ ------ ------ ---------
       19.06598        20.061134         17.98525 16.697 15.994 15.868 271970634
      19.486813        20.899199         18.17955 16.568 15.761 15.252 271273659
      18.801517        19.611595        17.864454 16.587 15.862 15.314 271981619
      17.345057        17.782911         16.73697 15.999 15.509 15.318 271223089
      18.668016        19.498903        17.653946 16.393 15.711 15.498 271208327
      16.346586         16.66519         15.86856 15.294 15.064 14.885 271919488
      15.356233        15.818361        14.730853 13.953 13.453 13.377 271965018
      18.073174         18.70632        17.315163  16.36 15.802 15.282 271944076
      18.862864         20.19449        17.644482 15.971 15.529 15.401 272134874
       15.78973        16.21

# Try GAIA and WISE

In [54]:
cmd = "SELECT * FROM gaiadr2.gaia_source AS g, \
gaiadr2.allwise_best_neighbour AS wbest, \
gaiadr1.allwise_original_valid AS allwise \
WHERE g.source_id = wbest.source_id AND wbest.allwise_oid = allwise.allwise_oid AND CONTAINS(POINT('ICRS',g.ra,g.dec),\
CIRCLE('ICRS'," + str(obj_ra) + "," + str(obj_dec) + "," + str(cone_radius) + "))=1;"

print(cmd)


job = Gaia.launch_job_async(cmd, dump_to_file=True)
print(job)

SELECT * FROM gaiadr2.gaia_source AS g, gaiadr2.allwise_best_neighbour AS wbest, gaiadr1.allwise_original_valid AS allwise WHERE g.source_id = wbest.source_id AND wbest.allwise_oid = allwise.allwise_oid AND CONTAINS(POINT('ICRS',g.ra,g.dec),CIRCLE('ICRS',1.55789,-6.39315,1))=1;
Jobid: 1542985794627O
Phase: COMPLETED
Owner: None
Output file: async_20181123120954.vot
Results: None


In [55]:
r = job.get_results()
print(len(r['source_id']))

6152


In [58]:
print(r['w1mpro', 'w2mpro', 'w3mpro', 'w4mpro']) 

w1mpro w2mpro w3mpro w4mpro
 mag    mag    mag    mag  
------ ------ ------ ------
15.766 15.452 11.957   8.91
15.922 15.838  12.46  8.822
16.123 15.152 12.408  8.396
15.369 15.248 11.925  8.983
15.798 15.462  11.91  8.776
 15.51 15.567 12.442  9.049
15.331 15.483 11.678  8.909
14.854 14.875 12.385  8.721
16.925 16.794 12.452  8.799
13.355 13.393  11.93  9.003
   ...    ...    ...    ...
15.359 15.442 12.364  8.839
15.481 15.436 12.206   8.99
13.177 13.201 12.346  8.767
14.993 14.874 11.824   8.97
11.363 11.362 11.735  8.306
14.932  15.03  12.47  8.785
15.519 15.373 12.242  8.563
15.736 15.711 12.458  9.077
15.918  15.89 12.268  8.849
16.654 16.701 12.389  8.602
Length = 6152 rows


## Try GAIA + WISE with ProperMotion limit

In [5]:
cmd = "SELECT * FROM gaiadr2.gaia_source AS g, \
gaiadr2.allwise_best_neighbour AS wbest, \
gaiadr1.allwise_original_valid AS allwise \
WHERE g.source_id = wbest.source_id AND wbest.allwise_oid = allwise.allwise_oid AND CONTAINS(POINT('ICRS',g.ra,g.dec),\
CIRCLE('ICRS'," + str(obj_ra) + "," + str(obj_dec) + "," + str(cone_radius) + "))=1 \
AND pmra IS NOT NULL AND abs(pmra)<10 \
AND pmdec IS NOT NULL AND abs(pmdec)<10;"

print(cmd)


job = Gaia.launch_job_async(cmd, dump_to_file=True)
print(job)

SELECT * FROM gaiadr2.gaia_source AS g, gaiadr2.allwise_best_neighbour AS wbest, gaiadr1.allwise_original_valid AS allwise WHERE g.source_id = wbest.source_id AND wbest.allwise_oid = allwise.allwise_oid AND CONTAINS(POINT('ICRS',g.ra,g.dec),CIRCLE('ICRS',1.55789,-6.39315,1))=1 AND pmra IS NOT NULL AND abs(pmra)<10 AND pmdec IS NOT NULL AND abs(pmdec)<10;
Jobid: 1543001175644O
Phase: COMPLETED
Owner: None
Output file: async_20181123162615.vot
Results: None


In [6]:
r = job.get_results()
print(len(r['source_id']))

2361


In [7]:
print(r['pmra', 'pmdec', 'w1mpro']) 

        pmra               pmdec        w1mpro
     mas.yr**-1          mas.yr**-1      mag  
------------------- ------------------- ------
-0.2580540328041185  -4.303071386306459  15.51
 3.2412908910044624  -6.625175072390147 15.331
   4.81042373536586 -2.7747316177439822 14.854
 -2.343859986856239 -2.7118726610965034 16.925
 -6.518195548921061  -1.008092891140326 13.355
  7.919036899673606 -1.5511593966580661 16.338
 -4.096934742443867   -7.95511552556917 16.299
  5.250329290337021  -2.621035875459965 15.543
  -6.91772926918492  -7.702118666055642 16.582
  4.545161908035533 -1.8590536895464917 15.123
                ...                 ...    ...
 1.0373230064566457  0.2406560377003164 14.595
 -3.498653090694471 -1.2743232598233822 15.624
 0.5905450541204029  0.2104775787639945 16.828
 -6.109687397707301  -6.834969854651878 16.015
  -6.76713721701586 -1.6416193859466321 14.118
-1.2892615805163268  -5.759760884707478   14.8
  8.364772083438973   -5.38598792104008 16.238
  3.710599758

# Try GAIA-WISE-2MASS directly

- I check there is tmass_key in gaiadr1.allwise_original_valid (ALLWISE catalog)

Desc of tmass_key:

2MASS PSC association. Unique identifier of the closest source in the 2MASS Point Source Catalog (PSC) that falls within 3 arcsec of the non-motion fit position of this WISE source. This is equivalent to the pts_key in the 2MASS PSC entry. This column is “null” if there is no 2MASS PSC source within 3 arcsec of the WISE source position.

In [90]:
cmd = "SELECT * FROM gaiadr2.gaia_source AS g, \
gaiadr2.allwise_best_neighbour AS wbest, \
gaiadr1.allwise_original_valid AS allwise, \
gaiadr1.tmass_original_valid AS tmass \
WHERE g.source_id = wbest.source_id AND wbest.allwise_oid = allwise.allwise_oid AND CONTAINS(POINT('ICRS',g.ra,g.dec),\
CIRCLE('ICRS'," + str(obj_ra) + "," + str(obj_dec) + "," + str(cone_radius) + "))=1\
AND allwise.tmass_key IS NOT NULL \
AND allwise.tmass_key = tmass.tmass_oid;"

print(cmd)


job = Gaia.launch_job_async(cmd, dump_to_file=True)
print(job)

SELECT * FROM gaiadr2.gaia_source AS g, gaiadr2.allwise_best_neighbour AS wbest, gaiadr1.allwise_original_valid AS allwise, gaiadr1.tmass_original_valid AS tmass WHERE g.source_id = wbest.source_id AND wbest.allwise_oid = allwise.allwise_oid AND CONTAINS(POINT('ICRS',g.ra,g.dec),CIRCLE('ICRS',1.55789,-6.39315,1))=1AND allwise.tmass_key IS NOT NULL AND allwise.tmass_key = tmass.tmass_oid;
Jobid: 1542989695550O
Phase: COMPLETED
Owner: None
Output file: async_20181123131455.vot
Results: None


In [91]:
r = job.get_results()
print(len(r['source_id']))

2990


In [8]:
print(r.colnames)

['solution_id', 'designation', 'source_id', 'random_index', 'ref_epoch', 'ra', 'ra_error', 'dec', 'dec_error', 'parallax', 'parallax_error', 'parallax_over_error', 'pmra', 'pmra_error', 'pmdec', 'pmdec_error', 'ra_dec_corr', 'ra_parallax_corr', 'ra_pmra_corr', 'ra_pmdec_corr', 'dec_parallax_corr', 'dec_pmra_corr', 'dec_pmdec_corr', 'parallax_pmra_corr', 'parallax_pmdec_corr', 'pmra_pmdec_corr', 'astrometric_n_obs_al', 'astrometric_n_obs_ac', 'astrometric_n_good_obs_al', 'astrometric_n_bad_obs_al', 'astrometric_gof_al', 'astrometric_chi2_al', 'astrometric_excess_noise', 'astrometric_excess_noise_sig', 'astrometric_params_solved', 'astrometric_primary_flag', 'astrometric_weight_al', 'astrometric_pseudo_colour', 'astrometric_pseudo_colour_error', 'mean_varpi_factor_al', 'astrometric_matched_observations', 'visibility_periods_used', 'astrometric_sigma5d_max', 'frame_rotator_object_type', 'matched_observations', 'duplicated_source', 'phot_g_n_obs', 'phot_g_mean_flux', 'phot_g_mean_flux_erro

In [93]:
r['ra', 'dec', 'ra_2', 'dec_2', 'ra_3', 'dec_3', 'phot_g_mean_mag', 'j_m', 'w1mpro', 'tmass_key', 'tmass_oid']

ra,dec,ra_2,dec_2,ra_3,dec_3,phot_g_mean_mag,j_m,w1mpro,tmass_key,tmass_oid
deg,deg,deg,deg,deg,deg,mag,mag,mag,,
float64,float64,float64,float64,float64,float64,float32,float32,float64,int64,int64
2.390732751830467,-6.546052592994798,2.3906145,-6.5460367,252.701422,-42.584721,19.06598,14.96,15.766,125705984,125705984
2.4119940994386777,-6.754831866119,2.411886,-6.7547434,146.318281,-42.584644,19.486813,16.38,15.369,125706276,125706276
2.5428858650859794,-6.5427542995979895,2.5428842,-6.5427375,240.113163,-42.580746,18.801517,16.472,15.798,125722213,125722213
2.4657746243804772,-6.770026089663108,2.465796,-6.7700469,255.438205,-42.580818,17.345057,13.86,15.51,125721895,125721895
2.3905290941368316,-6.77431073539794,2.3905771,-6.7742426,259.178725,-42.584637,18.668016,11.326,15.331,125706301,125706301
2.3086280130046433,-6.561269746375928,2.3086345,-6.5612655,267.035809,-42.585201,16.346586,16.63,14.854,125704003,125704003
2.5481475582031994,-6.547613323459481,2.5481413,-6.5476079,275.870923,-42.580746,15.356233,16.304,13.355,125722207,125722207
2.471145860590163,-6.553873161367393,2.4711055,-6.5537591,253.782353,-42.58075,18.073174,16.214,15.543,125722199,125722199


## tmass_key is NOT the same with tmass_oid!